# 🧹 SILVER LAYER - Transformation et Nettoyage

**Flux ETL** : Bronze (brut) → **Silver (nettoyé)** → Gold (métier)

**Auteurs** : Nejma MOUALHI | Brieuc OLIVIERI | Nicolas TAING

---

## 🎯 Transformations appliquées

### 1. ANONYMISATION (RGPD) :
- ✅ Hash des noms/prénoms (SHA-256)
- ✅ Suppression des données sensibles

### 2. NETTOYAGE :
- ✅ Formats de dates cohérents (YYYY-MM-DD)
- ✅ Typage correct des colonnes
- ✅ Dédoublonnage
- ✅ Valeurs NULL gérées

### 3. VALIDATION :
- ✅ Contraintes métier vérifiées
- ✅ Données aberrantes filtrées

In [11]:
# Imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, sha2, when, trim, upper, 
    to_date, year, month, dayofmonth,
    regexp_replace, coalesce, lit,
    current_timestamp
)
from datetime import datetime
import time

print("✅ Imports OK")

✅ Imports OK


In [12]:
# Configuration Spark
spark = SparkSession.builder \
    .appName("CHU_Silver_Transform_Nettoyage") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

print(f"✅ Spark {spark.version} démarré")

# Chemins
BRONZE_BASE = "/home/jovyan/data/bronze"
SILVER_BASE = "/home/jovyan/data/silver"

print(f"📖 Source: {BRONZE_BASE}")
print(f"💾 Destination: {SILVER_BASE}")

✅ Spark 3.5.0 démarré
📖 Source: /home/jovyan/data/bronze
💾 Destination: /home/jovyan/data/silver


## 🔐 PARTIE 1 : Anonymisation Patient

In [13]:
print("="*80)
print("🔐 TRANSFORMATION: Patient (ANONYMISATION)")
print("="*80)

start_time = time.time()

# Lecture Bronze
df_patient_bronze = spark.read.parquet(f"{BRONZE_BASE}/postgres/Patient")
print(f"📖 Lu: {df_patient_bronze.count():,} lignes")

# Aperçu AVANT anonymisation
print("\n👀 AVANT anonymisation:")
df_patient_bronze.select("Id_patient", "Nom", "Prenom", "Sexe", "Age", "Date").show(3, truncate=False)

# ANONYMISATION + NETTOYAGE
df_patient_silver = df_patient_bronze.select(
    col("Id_patient").alias("id_patient"),
    
    # ANONYMISATION: Hash SHA-256 des données sensibles
    sha2(col("Nom"), 256).alias("nom_hash"),
    sha2(col("Prenom"), 256).alias("prenom_hash"),
    
    # Données démographiques (conservées)
    col("Sexe").alias("sexe"),
    col("Age").cast("integer").alias("age"),
    
    # FORMAT DATE UNIFORME: M/d/yyyy → yyyy-MM-dd
    to_date(col("Date"), "M/d/yyyy").alias("date_naissance"),
    
    # Localisation (géographique large = OK pour RGPD)
    trim(upper(col("Ville"))).alias("ville"),
    col("Code_postal").alias("code_postal"),
    trim(upper(col("Pays"))).alias("pays"),
    
    # Informations médicales
    col("Poid").cast("double").alias("poids_kg"),  # Correction: "Poid" pas "Poids"
    col("Taille").cast("double").alias("taille_cm"),
    trim(upper(col("Groupe_sanguin"))).alias("groupe_sanguin"),
    
    # Contact (hashé)
    sha2(col("Tel"), 256).alias("telephone_hash"),  # Correction: "Tel" pas "Telephone"
    sha2(col("EMail"), 256).alias("email_hash"),     # Correction: "EMail" pas "Email"
    
    # Sécurité sociale (hashée)
    sha2(col("Num_Secu"), 256).alias("num_secu_hash"),  # Correction: "Num_Secu" pas "Numero_Securite_sociale"
    
    # Métadonnées
    col("ingestion_date"),
    current_timestamp().alias("transformation_timestamp")
).dropDuplicates(["id_patient"])  # Dédoublonnage

# Aperçu APRÈS anonymisation
print("\n🔒 APRÈS anonymisation:")
df_patient_silver.select(
    "id_patient", "nom_hash", "prenom_hash", "sexe", "age", "date_naissance", "ville"
).show(3, truncate=False)

# Sauvegarde Silver
df_patient_silver.write \
    .mode("overwrite") \
    .parquet(f"{SILVER_BASE}/patient")

elapsed = time.time() - start_time
print(f"\n💾 Sauvegardé: {SILVER_BASE}/patient")
print(f"⏱️  Temps: {elapsed:.2f}s")
print(f"✅ {df_patient_silver.count():,} patients anonymisés")

🔐 TRANSFORMATION: Patient (ANONYMISATION)
📖 Lu: 100,000 lignes

👀 AVANT anonymisation:
+----------+----------+------+------+---+---------+
|Id_patient|Nom       |Prenom|Sexe  |Age|Date     |
+----------+----------+------+------+---+---------+
|1         |Christabel|Tougas|female|41 |4/6/1980 |
|2         |Lorraine  |Lebel |female|7  |7/25/2013|
|3         |Jolie     |Majory|female|11 |8/8/2009 |
+----------+----------+------+------+---+---------+
only showing top 3 rows


🔒 APRÈS anonymisation:
+----------+----------------------------------------------------------------+----------------------------------------------------------------+------+---+--------------+-----------------+
|id_patient|nom_hash                                                        |prenom_hash                                                     |sexe  |age|date_naissance|ville            |
+----------+----------------------------------------------------------------+-------------------------------------------------

## 📅 PARTIE 2 : Nettoyage Consultation (Dates + Typage)

In [14]:
print("="*80)
print("📅 TRANSFORMATION: Consultation (DATES + TYPAGE)")
print("="*80)

start_time = time.time()

# Lecture Bronze
df_consult_bronze = spark.read.parquet(f"{BRONZE_BASE}/postgres/Consultation")
print(f"📖 Lu: {df_consult_bronze.count():,} lignes")

# NETTOYAGE + FORMATS
df_consult_silver = df_consult_bronze.select(
    col("Num_consultation").alias("id_consultation"),  # Correction: "Num_consultation"
    col("Id_patient").alias("id_patient"),
    col("Code_diag").alias("id_diagnostic"),           # Correction: "Code_diag"
    col("Id_prof_sante").alias("id_professionnel"),   # Correction: "Id_prof_sante"
    col("Id_mut").alias("id_mutuelle"),               # Ajout: colonne mutuelle disponible
    
    # FORMAT DATE: M/d/yyyy → yyyy-MM-dd
    to_date(col("Date"), "M/d/yyyy").alias("date_consultation"),
    
    # Extraction composantes temporelles (pour partitionnement futur)
    year(to_date(col("Date"), "M/d/yyyy")).alias("annee"),
    month(to_date(col("Date"), "M/d/yyyy")).alias("mois"),
    dayofmonth(to_date(col("Date"), "M/d/yyyy")).alias("jour"),
    
    # Heures de début et fin (disponibles dans les données)
    col("Heure_debut").alias("heure_debut"),
    col("Heure_fin").alias("heure_fin"),
    
    # Motif de consultation
    trim(col("Motif")).alias("motif"),
    
    # Métadonnées
    col("ingestion_date"),
    current_timestamp().alias("transformation_timestamp")
).filter(
    # VALIDATION: dates cohérentes (2013-2025)
    (col("annee") >= 2013) & (col("annee") <= 2025)
).dropDuplicates(["id_consultation"])

# Aperçu
print("\n📊 Données nettoyées:")
df_consult_silver.select(
    "id_consultation", "date_consultation", "annee", "mois", "heure_debut", "motif"
).show(5)

# Sauvegarde Silver
df_consult_silver.write \
    .mode("overwrite") \
    .parquet(f"{SILVER_BASE}/consultation")

elapsed = time.time() - start_time
print(f"\n💾 Sauvegardé: {SILVER_BASE}/consultation")
print(f"⏱️  Temps: {elapsed:.2f}s")
print(f"✅ {df_consult_silver.count():,} consultations nettoyées")

📅 TRANSFORMATION: Consultation (DATES + TYPAGE)
📖 Lu: 1,027,157 lignes

📊 Données nettoyées:
+---------------+-----------------+-----+----+-------------------+---------------+
|id_consultation|date_consultation|annee|mois|        heure_debut|          motif|
+---------------+-----------------+-----+----+-------------------+---------------+
|     1059023437|       2015-06-20| 2015|   6|1970-01-01 13:00:00|   Consultation|
|     1059023446|       2015-06-20| 2015|   6|1970-01-01 08:00:00|   Consultation|
|     1059023466|       2015-06-20| 2015|   6|1970-01-01 08:00:00|Soins dentaires|
|     1059023468|       2015-06-20| 2015|   6|1970-01-01 13:00:00|   Consultation|
|     1059023498|       2015-06-20| 2015|   6|1970-01-01 13:00:00|   Consultation|
+---------------+-----------------+-----+----+-------------------+---------------+
only showing top 5 rows


💾 Sauvegardé: /home/jovyan/data/silver/consultation
⏱️  Temps: 8.69s
✅ 1,027,157 consultations nettoyées


## 🏥 PARTIE 3 : Nettoyage Établissements (CSV)

In [15]:
print("="*80)
print("🏥 TRANSFORMATION: Établissements de santé (CSV)")
print("="*80)

start_time = time.time()

# Lecture Bronze
df_etab_bronze = spark.read.parquet(f"{BRONZE_BASE}/csv/etablissement_sante")
print(f"📖 Lu: {df_etab_bronze.count():,} lignes")

# NETTOYAGE
df_etab_silver = df_etab_bronze.select(
    # Identifiants
    trim(col("finess_site")).alias("finess_site"),
    trim(col("siret_site")).alias("siret_site"),
    trim(col("siren_site")).alias("siren_site"),
    
    # Nom
    trim(col("raison_sociale_site")).alias("raison_sociale"),
    trim(col("enseigne_commerciale_site")).alias("enseigne_commerciale"),
    
    # Adresse (normalisée)
    trim(col("numero_voie")).alias("numero_voie"),
    trim(upper(col("type_voie"))).alias("type_voie"),
    trim(upper(col("voie"))).alias("voie"),
    trim(col("code_postal")).alias("code_postal"),
    trim(upper(col("commune"))).alias("commune"),
    trim(col("cedex")).alias("cedex"),
    trim(upper(col("pays"))).alias("pays"),
    
    # Contact (nettoyé, PAS hashé car données publiques)
    regexp_replace(col("telephone"), "[^0-9]", "").alias("telephone"),
    trim(col("email")).alias("email"),
    
    # Métadonnées
    col("ingestion_date"),
    current_timestamp().alias("transformation_timestamp")
).filter(
    # VALIDATION: au moins un identifiant présent
    col("finess_site").isNotNull() | col("siret_site").isNotNull()
).dropDuplicates(["finess_site", "siret_site"])

# Aperçu
print("\n📊 Données nettoyées:")
df_etab_silver.select(
    "finess_site", "raison_sociale", "commune", "code_postal"
).show(5, truncate=False)

# Sauvegarde Silver
df_etab_silver.write \
    .mode("overwrite") \
    .parquet(f"{SILVER_BASE}/etablissement_sante")

elapsed = time.time() - start_time
print(f"\n💾 Sauvegardé: {SILVER_BASE}/etablissement_sante")
print(f"⏱️  Temps: {elapsed:.2f}s")
print(f"✅ {df_etab_silver.count():,} établissements nettoyés")

🏥 TRANSFORMATION: Établissements de santé (CSV)
📖 Lu: 416,665 lignes

📊 Données nettoyées:
+-----------+-----------------------------------------+-------+-----------+
|finess_site|raison_sociale                           |commune|code_postal|
+-----------+-----------------------------------------+-------+-----------+
|NULL       |EURODOC SRL                              |NULL   |18039      |
|NULL       |DIRECTION GENERALE DE LA POLICE NATIONALE|PARIS  |75008      |
|NULL       |DIRECTION GENERALE DE L AVIATION CIVILE  |PARIS  |75720      |
|NULL       |MDPH DU MAINE ET LOIRE                   |ANGERS |49100      |
|NULL       |MDPH DE L'AUBE                           |TROYES |10026      |
+-----------+-----------------------------------------+-------+-----------+
only showing top 5 rows


💾 Sauvegardé: /home/jovyan/data/silver/etablissement_sante
⏱️  Temps: 4.14s
✅ 72,017 établissements nettoyés


## ⭐ PARTIE 4 : Nettoyage Satisfaction (CSV)

In [16]:
print("="*80)
print("⭐ TRANSFORMATION: Satisfaction 2019 (CSV)")
print("="*80)

start_time = time.time()

# Lecture Bronze
df_satis_bronze = spark.read.parquet(f"{BRONZE_BASE}/csv/satisfaction_esatis48h_2019")
print(f"📖 Lu: {df_satis_bronze.count():,} lignes")

# NETTOYAGE + TYPAGE
df_satis_silver = df_satis_bronze.select(
    # Identifiants
    trim(col("finess")).alias("finess"),
    trim(col("finess_geo")).alias("finess_geo"),
    trim(col("region")).alias("region"),
    
    # Noms
    trim(col("rs_finess")).alias("raison_sociale_finess"),
    
    # Participation
    trim(col("participation")).alias("participation"),
    trim(col("depot")).alias("depot"),
    
    # Scores (conversion en DOUBLE)
    col("score_all_rea_ajust").cast("double").alias("score_global"),
    col("score_accueil_rea_ajust").cast("double").alias("score_accueil"),
    col("score_PECinf_rea_ajust").cast("double").alias("score_pec_infirmier"),
    col("score_PECmed_rea_ajust").cast("double").alias("score_pec_medical"),
    col("score_chambre_rea_ajust").cast("double").alias("score_chambre"),
    col("score_repas_rea_ajust").cast("double").alias("score_repas"),
    col("score_sortie_rea_ajust").cast("double").alias("score_sortie"),
    
    # Nombre réponses (conversion en INTEGER)
    col("nb_rep_score_all_rea_ajust").cast("integer").alias("nb_reponses_global"),
    col("nb_reco_brut").cast("integer").alias("nb_recommandations"),
    
    # Taux recommandation
    col("taux_reco_brut").cast("double").alias("taux_recommandation"),
    
    # Classement
    trim(col("classement")).alias("classement"),
    trim(col("evolution")).alias("evolution"),
    
    # Année fixe
    lit(2019).alias("annee"),
    
    # Métadonnées
    col("ingestion_date"),
    current_timestamp().alias("transformation_timestamp")
).filter(
    # VALIDATION: score global présent
    col("score_global").isNotNull()
).dropDuplicates(["finess"])

# Aperçu
print("\n📊 Données nettoyées:")
df_satis_silver.select(
    "finess", "score_global", "score_accueil", "nb_reponses_global", "classement"
).show(5)

# Sauvegarde Silver
df_satis_silver.write \
    .mode("overwrite") \
    .parquet(f"{SILVER_BASE}/satisfaction_2019")

elapsed = time.time() - start_time
print(f"\n💾 Sauvegardé: {SILVER_BASE}/satisfaction_2019")
print(f"⏱️  Temps: {elapsed:.2f}s")
print(f"✅ {df_satis_silver.count():,} évaluations nettoyées")

⭐ TRANSFORMATION: Satisfaction 2019 (CSV)
📖 Lu: 1,152 lignes

📊 Données nettoyées:
+---------+------------+-------------+------------------+----------+
|   finess|score_global|score_accueil|nb_reponses_global|classement|
+---------+------------+-------------+------------------+----------+
|070780358|        71.0|        71.15|               307|         C|
|180000358|        76.0|        78.05|                78|         B|
|380785956|        77.0|        77.35|              1045|         B|
|640018206|        76.0|        75.57|               326|         B|
|670780055|        73.0|        74.35|               987|         C|
+---------+------------+-------------+------------------+----------+
only showing top 5 rows


💾 Sauvegardé: /home/jovyan/data/silver/satisfaction_2019
⏱️  Temps: 1.23s
✅ 8 évaluations nettoyées


## 💀 PARTIE 5 : Anonymisation Décès (CSV)

In [17]:
print("="*80)
print("💀 TRANSFORMATION: Décès 2019 (ANONYMISATION)")
print("="*80)

start_time = time.time()

# Lecture Bronze
df_deces_bronze = spark.read.parquet(f"{BRONZE_BASE}/csv/deces_2019")
print(f"📖 Lu: {df_deces_bronze.count():,} lignes")

# ANONYMISATION + NETTOYAGE
df_deces_silver = df_deces_bronze.select(
    # ANONYMISATION: Hash des identités
    sha2(col("nom"), 256).alias("nom_hash"),
    sha2(col("prenom"), 256).alias("prenom_hash"),
    sha2(col("numero_acte_deces"), 256).alias("acte_deces_hash"),
    
    # Données démographiques
    col("sexe"),
    to_date(col("date_naissance")).alias("date_naissance"),
    to_date(col("date_deces")).alias("date_deces"),
    
    # Âge au décès (calculé)
    (year(to_date(col("date_deces"))) - year(to_date(col("date_naissance")))).alias("age_deces"),
    
    # Lieux (géographiques larges = OK RGPD)
    trim(col("code_lieu_naissance")).alias("code_lieu_naissance"),
    trim(upper(col("lieu_naissance"))).alias("lieu_naissance"),
    trim(upper(col("pays_naissance"))).alias("pays_naissance"),
    
    trim(col("code_lieu_deces")).alias("code_lieu_deces"),
    
    # Année/mois pour agrégations
    year(to_date(col("date_deces"))).alias("annee_deces"),
    month(to_date(col("date_deces"))).alias("mois_deces"),
    
    # Métadonnées
    col("ingestion_date"),
    current_timestamp().alias("transformation_timestamp")
).filter(
    # VALIDATION: dates cohérentes
    (col("date_deces").isNotNull()) & 
    (col("annee_deces") == 2019) &
    (col("age_deces") >= 0) & (col("age_deces") <= 120)
)

# Aperçu
print("\n📊 Données anonymisées:")
df_deces_silver.select(
    "sexe", "age_deces", "date_deces", "lieu_naissance", "code_lieu_deces"
).show(5, truncate=False)

# Sauvegarde Silver
df_deces_silver.write \
    .mode("overwrite") \
    .parquet(f"{SILVER_BASE}/deces_2019")

elapsed = time.time() - start_time
print(f"\n💾 Sauvegardé: {SILVER_BASE}/deces_2019")
print(f"⏱️  Temps: {elapsed:.2f}s")
print(f"✅ {df_deces_silver.count():,} décès anonymisés")

💀 TRANSFORMATION: Décès 2019 (ANONYMISATION)
📖 Lu: 620,626 lignes

📊 Données anonymisées:
+----+---------+----------+---------------+---------------+
|sexe|age_deces|date_deces|lieu_naissance |code_lieu_deces|
+----+---------+----------+---------------+---------------+
|2   |93       |2019-01-29|VENISSIEUX     |01004          |
|1   |57       |2019-01-11|BOURGES        |01004          |
|1   |88       |2019-01-19|MACON          |01004          |
|1   |74       |2019-01-20|BOURG-EN-BRESSE|01004          |
|1   |74       |2019-01-01|RUMILLY        |01004          |
+----+---------+----------+---------------+---------------+
only showing top 5 rows


💾 Sauvegardé: /home/jovyan/data/silver/deces_2019
⏱️  Temps: 6.22s
✅ 620,625 décès anonymisés


## 🏥 PARTIE 6 : Tables de référence (simples)

In [18]:
# Tables de référence (pas d'anonymisation nécessaire)
reference_tables = [
    "Diagnostic",
    "Professionnel_de_sante",
    "Mutuelle",
    "Medicaments",
    "Laboratoire",
    "Salle",
    "Specialites"
]

print("="*80)
print("📋 TRANSFORMATION: Tables de référence")
print("="*80)

for table in reference_tables:
    print(f"\n🔄 {table}...")
    start_time = time.time()
    
    # Lecture Bronze
    df = spark.read.parquet(f"{BRONZE_BASE}/postgres/{table}")
    
    # Nettoyage basique: trim des strings, dédoublonnage
    df_clean = df
    for col_name in df.columns:
        if dict(df.dtypes)[col_name] == 'string':
            df_clean = df_clean.withColumn(col_name, trim(col(col_name)))
    
    # Ajout métadonnées
    df_clean = df_clean.withColumn(
        "transformation_timestamp", 
        current_timestamp()
    )
    
    # Sauvegarde
    output_path = f"{SILVER_BASE}/{table.lower()}"
    df_clean.write.mode("overwrite").parquet(output_path)
    
    elapsed = time.time() - start_time
    print(f"   ✅ {df_clean.count():,} lignes - {elapsed:.2f}s")

📋 TRANSFORMATION: Tables de référence

🔄 Diagnostic...
   ✅ 15,490 lignes - 0.90s

🔄 Professionnel_de_sante...
   ✅ 1,048,575 lignes - 3.66s

🔄 Mutuelle...
   ✅ 254 lignes - 0.77s

🔄 Medicaments...
   ✅ 15,455 lignes - 0.87s

🔄 Laboratoire...
   ✅ 677 lignes - 0.74s

🔄 Salle...
   ✅ 201,735 lignes - 1.20s

🔄 Specialites...
   ✅ 93 lignes - 0.81s


## 📊 RÉSUMÉ SILVER LAYER

In [19]:
# Vérification finale
print("\n" + "="*80)
print("📊 RÉSUMÉ SILVER LAYER")
print("="*80)

silver_tables = [
    "patient",
    "consultation",
    "etablissement_sante",
    "satisfaction_2019",
    "deces_2019",
    "diagnostic",
    "professionnel_de_sante",
    "mutuelle",
    "medicaments",
    "laboratoire",
    "salle",
    "specialites"
]

total_rows = 0
for table in silver_tables:
    try:
        df = spark.read.parquet(f"{SILVER_BASE}/{table}")
        count = df.count()
        total_rows += count
        print(f"  ✅ {table:30} {count:>10,} lignes")
    except Exception as e:
        print(f"  ❌ {table:30} ERREUR")

print("="*80)
print(f"\n📊 TOTAL SILVER: {total_rows:,} lignes")
print(f"💾 Stockage: {SILVER_BASE}/")
print("="*80)


📊 RÉSUMÉ SILVER LAYER
  ✅ patient                           100,000 lignes
  ✅ consultation                    1,027,157 lignes
  ✅ etablissement_sante                72,017 lignes
  ✅ satisfaction_2019                       8 lignes
  ✅ deces_2019                        620,625 lignes
  ✅ diagnostic                         15,490 lignes
  ✅ professionnel_de_sante          1,048,575 lignes
  ✅ mutuelle                              254 lignes
  ✅ medicaments                        15,455 lignes
  ✅ laboratoire                           677 lignes
  ✅ salle                             201,735 lignes
  ✅ specialites                            93 lignes

📊 TOTAL SILVER: 3,102,086 lignes
💾 Stockage: /home/jovyan/data/silver/


---

## ✅ SILVER LAYER - TRANSFORMATIONS COMPLÈTES

### 🔐 Anonymisation (RGPD) :
- ✅ Patient : Noms/prénoms/contact hashés SHA-256
- ✅ Décès : Identités hashées

### 🧹 Nettoyage :
- ✅ Dates uniformisées (yyyy-MM-dd)
- ✅ Typage correct (integer, double, date)
- ✅ Normalisation texte (trim, upper)
- ✅ Dédoublonnage

### ✅ Validation :
- ✅ Contraintes métier vérifiées
- ✅ Valeurs aberrantes filtrées
- ✅ Identifiants présents

### 🎯 Prochaine étape :

👉 **Notebook 03** : Transform Gold (Star Schema + Optimisations)

**Important** : Les données Silver sont maintenant **propres**, **validées** et **anonymisées**. Prêtes pour le modèle métier Gold !